The NatureServe Explorer system contains a number of useful fields of information associated with sensitive species. The bispy package contains a module and set of functions for interacting with the NatureServe API. This notebook searches NatureServe and retrieves species documents for a cache to be used in further evaluation. The NatureServe API is a little bit old, and its XML response is challenging to work with. The Python function in bispy provides a little bit of reformatting into a dictionary object (JSON) for ease of processing.


# Data Management Considerations
Currently, we are simply working with the one public, open API for NatureServe explorer as we found it was seemingly more reliable than working with the services requiring an access key and contains enough information to get a start at working with what NatureServe has to offer. Private API routes also exist that require an API key for access. In either case, we comply with the posted [usage requirements](https://services.natureserve.org/idd/developer/license.jsp). Our team has had conversations with NatureServe staff and management about our particular usage of NatureServe data, verifying that we are using the information appropriately, including storing relevant "documents" associated with species in our lists, sharing selected information via our APIs, and displaying on web apps. The main concern that was expressed in discussions was that we always include the date that a NatureServe species record was last reviewed so that users can be aware of cases where they may be looking at outdated information.

In [1]:
import requests
import bispy
from IPython.display import display
from joblib import Parallel, delayed
from collections import Counter

natureserve = bispy.natureserve.Natureserve()
bis_utils = bispy.bis.Utils()

import helperfunctions

In [2]:
name_list = helperfunctions.workplan_species()

In [3]:
# Use joblib to run multiple requests for records in parallel via scientific names
natureserve_results = Parallel(n_jobs=8)(delayed(natureserve.search)(name, name_source) for name, name_source in name_list)


In [4]:
natureserve_results_filtered = [r for r in natureserve_results if r["Processing Metadata"]["Summary Result"] != "Not Matched"]

In [5]:
# Cache the array of retrieved documents and return/display a random sample for verification
display(bis_utils.doc_cache("cache/natureserve.json", natureserve_results_filtered))


{'Doc Cache File': 'cache/natureserve.json',
 'Document Number 270': {'NatureServe Species': {'@type': 'Animal',
   '@uid': 'ELEMENT_NATIONAL.2.170522',
   'jurisdictionNationName': {'#text': 'UNITED STATES', '@code': 'US'},
   'metadata': {'references': None},
   'nationalCommonName': 'Narrow-foot Hydrotus Diving Beetle',
   'nationalConservationStatus': {'#text': 'N1N2',
    '@lastChangedDate': '2004-02-23',
    '@lastReviewedDate': '2004-02-23'},
   'nationalDistributions': {'nationalDistribution': {'currentPresenceAbsence': 'Present',
     'distributionConfidence': 'Confident',
     'origin': 'Native',
     'population': 'Year-round',
     'regularity': 'Regularly occurring'}},
   'nationalScientificName': {'formattedName': '<i>Hygrotus diversipes</i>',
    'nomenclaturalAuthor': 'Leech, 1966',
    'unformattedName': 'Hygrotus diversipes'},
   'natureServeGlobalConcept': {'@uid': 'ELEMENT_GLOBAL.2.108668',
    'classificationStatus': 'Standard',
    'globalConservationStatus': 'G1G

In [6]:
Counter(spp["NatureServe Species"]["roundedNationalConservationStatus"] for spp in natureserve_results_filtered)

Counter({'N1': 154,
         'N2': 102,
         'N3': 66,
         'N3B,N3N': 1,
         'N4': 10,
         'N4B': 2,
         'N4B,N4N': 1,
         'N5': 6,
         'NNR': 7,
         'NNRN': 1,
         'NU': 1})

In [7]:
Counter(spp["Processing Metadata"]["Name Source"] for spp in natureserve_results_filtered)

Counter({'Lookup Name': 340, 'Valid ITIS Scientific Name': 11})